# LangChain 入門


In [ ]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env", override=True)

## Chat model


### ChatOpenAI の基本的な使い方


In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="gpt-5-nano",
    model_provider="openai",
    reasoning_effort="minimal",
)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！私はジョンと言います"),
    AIMessage("こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage("私の名前がわかりますか？"),
]

ai_message = model.invoke(messages)
print(ai_message.content)

### ストリーミング


In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="gpt-5-nano",
    model_provider="openai",
    reasoning_effort="minimal",
)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！"),
]

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

## Prompt template


### ChatPromptTemplate


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

prompt_value = prompt.invoke({"dish": "カレー"})
for message in prompt_value.messages:
    print(f"{message.type}: {message.content}")

### MessagesPlaceholder


In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="こんにちは！私はジョンと言います！"),
            AIMessage("こんにちは、ジョンさん！どのようにお手伝いできますか？"),
        ],
        "input": "私の名前が分かりますか？",
    }
)
for message in prompt_value.messages:
    print(f"{message.type}: {message.content}")


## Output parser


### StrOutputParser


In [ ]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="こんにちは。私はAIアシスタントです。")
output = output_parser.invoke(ai_message)
print(type(output))
print(output)

### with_structured_output を使った Python オブジェクトへの変換


In [ ]:
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


model = init_chat_model(
    model="gpt-5-nano",
    model_provider="openai",
    reasoning_effort="minimal",
)
structured_model = model.with_structured_output(Recipe)
result = structured_model.invoke("カレーのレシピを教えて")

print(type(result))
print(result)

## LangChain Expression Language（LCEL）の概要


### prompt と model の連鎖


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = init_chat_model(
    model="gpt-5-nano",
    model_provider="openai",
    reasoning_effort="minimal",
)

In [ ]:
chain = prompt | model

In [ ]:
ai_message = chain.invoke({"dish": "カレー"})
print(type(ai_message))
print(ai_message)

### StrOutputParser を連鎖に追加


In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
output = chain.invoke({"dish": "カレー"})
print(type(output))
print(output)

### with_structured_output を使う連鎖


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = init_chat_model(
    model="gpt-5-nano",
    model_provider="openai",
    reasoning_effort="minimal",
)

chain = prompt | model.with_structured_output(Recipe)

recipe = chain.invoke({"dish": "カレー"})
print(type(recipe))
print(recipe)

# LangSmith のセットアップとトレースの確認


In [ ]:
# 実行する前に、Notebook上で「Restart」を実行してください

from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="../.env", override=True)

print(os.environ["LANGCHAIN_API_KEY"][:3])
print(os.environ["LANGCHAIN_TRACING_V2"])
# "lsv"と"true"が表示されれば、環境変数に設定できています

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = init_chat_model(
    model="gpt-5-nano",
    model_provider="openai",
    reasoning_effort="minimal",
)

chain = prompt | model | StrOutputParser()
output = chain.invoke({"dish": "カレー"})
print(output)

# ここでLangSmithのトレースを確認してください

# RAG (検索拡張生成) の基礎


## LangChain での RAG の実装をステップバイステップで動かそう


### Document loader


In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(
    path="../tmp/langchain",
    glob="**/*.mdx",
    loader_cls=TextLoader,
)

raw_docs = loader.load()
print(len(raw_docs))

### Document transformer


In [ ]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

docs = text_splitter.split_documents(raw_docs)
print(len(docs))

### Embedding model


In [ ]:
from langchain.embeddings import init_embeddings

embeddings = init_embeddings(model="text-embedding-3-small", provider="openai")

In [ ]:
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"

vector = embeddings.embed_query(query)
print(len(vector))
print(vector)

### Vector store


In [ ]:
# 注意:
# このセルの処理は、大勢が同時に実行するとレートリミットのエラーになる可能性があります
# もしもエラーになった場合は、少し時間をおいてもう一度実行してください

from langchain_chroma import Chroma

db = Chroma.from_documents(docs, embeddings)

### Retriever


In [ ]:
retriever = db.as_retriever()

In [ ]:
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"

context_docs = retriever.invoke(query)
print(f"len = {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)

### LCEL を使った RAG の Chain の実装


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langsmith import traceable

prompt = ChatPromptTemplate.from_template('''
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = init_chat_model(
    model="gpt-5-nano",
    model_provider="openai",
    reasoning_effort="minimal",
)

In [ ]:
from langchain_core.output_parsers import StrOutputParser


@traceable
def invoke_rag(query: str) -> str:
    documents = retriever.invoke(query)
    chain = prompt | model | StrOutputParser()
    return chain.invoke({"question": query, "context": documents})


output = invoke_rag(query)
print(output)